# Speed Skydiving Analysis and Scoring

## Environment setup

In [2]:
import dateutil.parser
import math
import os
import os.path
import shutil

import pandas as pd


COURSE_START  = 2700.0
COURSE_END    = 1700.0
DATA_SOURCE   = 'pr3d4t0r'
DEG_IN_RAD    = math.pi/180.0
VALID_MSL     = 100.0 # STD
YEAR_PATH_TAG = '18-'

## FlySight data sources

Copy the FlySight directory of interest to this workspace, using the same directory naming conventions.  Then execute this cell to move the current files to the `DATA_SOURCE` directory for aggregated files analysis.

In [3]:
count = 0

for root, directories, files in os.walk('.'):
    if YEAR_PATH_TAG not in root:
        next
    else:
        baseName = root.replace('./', '').replace('-', '')
        for file in files:
            source      = os.path.join(root, file)
            fileName    = '20%s.%s' % (baseName, file.replace('-', ''))
            destination = os.path.join(DATA_SOURCE, fileName)
            count      += 1
            shutil.move(source, destination)

        shutil.rmtree(root)

print('Completed - moved %d files to the %s data source' % (count, DATA_SOURCE))

Completed - moved 0 files to the pr3d4t0r data source


## Top speed and pitch analysis on a single data file

User selects a valid FlySight data path and file name.  Call the `calculateSpeedAndPitch(fileName)` function with this file name.  The function produces:

* Classical mean speed calculated from all the discrete speeds sampled within the course
* USPA calculation ds/dt where dt<sub>competitionTime</sub> = t<sub>end</sub>-t<sub>start</sub>,
  and s<sub>courseLength</sub> = |s<sub>start</sub>-s<sub>end</sub>| for the first and last heights
  reported within the course window.
* Flight pitch in degrees at max speed
* Max speed
* Min speed

All speeds are reported in km/h.  Michael Cooper, USPA National Speed Skydiving Championship judged, explained
the "USPA calculation" as straight ds/dt in an email message from 04.Sep.2018.

### Explanation of the code in calculateSpeedAndPitchFrom(fileName)

1. Discard all source entries outside of the 2,700 to 1,700 m AGL course
1. Calculate the mean speed for the current jump (actual sum(N)/N and ds/dt for s0, s1
1. Resolve the max speed and pitch within the course
1. Return the results in a Series object, for later inclusion in a data frame with results from multiple jumps

In [6]:
def discardDataOutsideCourse(fullFlightData):
    height          = fullFlightData['hMSL', '(m)']
    descentVelocity = fullFlightData['velD', '(m/s)']
    
    return fullFlightData[(height <= COURSE_START) & (height >= COURSE_END) & (descentVelocity >= 0.0)]


def calculateCourseSpeedUsing(flightData):
    # TODO: figure out why pd.to_datetime() barfs
    startTime = dateutil.parser.parse(flightData['time', 'Unnamed: 0_level_1'].values[0])
    endTime   = dateutil.parser.parse(flightData['time', 'Unnamed: 0_level_1'].values[-1])

    startCourse = flightData['hMSL', '(m)'].values[0]
    endCourse   = flightData['hMSL', '(m)'].values[-1]

    competitionTime = (endTime-startTime).total_seconds()
    courseLength    = abs(endCourse-startCourse)

    return courseLength/competitionTime, competitionTime


def maxHorizontalSpeedFrom(flightData, maxVerticalSpeed):
    velN = flightData[flightData['velD', '(m/s)'] == maxVerticalSpeed]['velN', '(m/s)']
    velE = flightData[flightData['velD', '(m/s)'] == maxVerticalSpeed]['velE', '(m/s)']
    
    return math.sqrt(velN**2+velE**2)   # R vector


def calculateSpeedAndPitchFor(fileName):
    flightData  = discardDataOutsideCourse(pd.read_csv(fileName, header = [0, 1]))
    sampledVelD = flightData['velD', '(m/s)'].mean()
    courseVelD, \
    courseTime  = calculateCourseSpeedUsing(flightData)
    maxSpeed    = flightData['velD', '(m/s)'].max()
    minSpeed    = flightData['velD', '(m/s)'].min()
    
    pitchR = math.atan(maxSpeed/maxHorizontalSpeedFrom(flightData, maxSpeed))
    
    skydiveResults = pd.Series(
                        [
                            '2018-00-00T00:00:00.00',
                            3.6*sampledVelD,  # km/h; 3,600 seconds, 1,000 meters
                            3.6*courseVelD,
                            pitchR/DEG_IN_RAD,
                            3.6*maxSpeed,
                            3.6*minSpeed,
                            courseTime,
                        ],
                        [
                            'competitionTime',
                            'sampledSpeed',
                            'courseSpeed',
                            'pitch',
                            'maxSpeed',
                            'minSpeed',
                            'courseTime',
                        ])

    return skydiveResults


FLYSIGHT_DATA_FILE = 'pr3d4t0r/20181112.122617.CSV'
calculateSpeedAndPitchFor(FLYSIGHT_DATA_FILE)

competitionTime    2018-00-00T00:00:00.00
sampledSpeed                      405.254
courseSpeed                       405.769
pitch                             72.0859
maxSpeed                          415.116
minSpeed                          397.836
courseTime                            8.6
dtype: object

## Listing FlySight data files with valid data

1. Generate the list of available data files
1. Discard FlySight warm up files by rejecting files without jump data (test:  minimal altitude changes)

In [39]:
def listDataFilesIn(path):
    """
    Generate a sorted list of files available in a given path.
    Files names appear in reverse lexicographical order.
    """

    filesList = pd.Series([os.path.join(path, fileName) for fileName in sorted(os.listdir(path), reverse = True) if '.CSV' in fileName])
    
    return filesList;

In [40]:
def hasValidJumpData(fileName):
    flightData = pd.read_csv(fileName, header = [0, 1])['hMSL', '(m)']
    
    return flightData.std() >= VALID_MSL

In [41]:
def discardFlySightWarmUpFilesFrom(dataFiles):
    included = dataFiles.apply(hasValidJumpData)
    
    return pd.Series(dataFiles)[included]

In [44]:
dataFiles = discardFlySightWarmUpFilesFrom(listDataFilesIn(DATA_SOURCE))
dataFiles

0     pr3d4t0r/20181112.131849.CSV
1     pr3d4t0r/20181112.122617.CSV
2     pr3d4t0r/20181112.111218.CSV
4     pr3d4t0r/20181104.173954.CSV
5     pr3d4t0r/20181104.170330.CSV
6     pr3d4t0r/20181104.154616.CSV
7     pr3d4t0r/20181027.155145.CSV
8     pr3d4t0r/20181027.142406.CSV
9     pr3d4t0r/20181027.125241.CSV
10    pr3d4t0r/20181027.115951.CSV
11    pr3d4t0r/20181027.103429.CSV
13    pr3d4t0r/20181020.105547.CSV
15    pr3d4t0r/20181014.150818.CSV
16    pr3d4t0r/20181014.122659.CSV
17    pr3d4t0r/20181013.140533.CSV
18    pr3d4t0r/20181013.121702.CSV
19    pr3d4t0r/20181013.110513.CSV
20    pr3d4t0r/20180922.172522.CSV
21    pr3d4t0r/20180922.154418.CSV
22    pr3d4t0r/20180922.140145.CSV
23    pr3d4t0r/20180916.164551.CSV
24    pr3d4t0r/20180916.154425.CSV
25    pr3d4t0r/20180916.145346.CSV
26    pr3d4t0r/20180916.132951.CSV
27    pr3d4t0r/20180916.121821.CSV
28    pr3d4t0r/20180916.103811.CSV
29    pr3d4t0r/20180916.092509.CSV
30    pr3d4t0r/20180916.072339.CSV
32    pr3d4t0r/20180